In [1]:
import model_based_mvpa as mbmvpa
from model_based_mvpa.preprocessing.bids import *
from model_based_mvpa.preprocessing.events import *
from model_based_mvpa.utils.func import *
from model_based_mvpa.models.model import *
from pathlib import Path
import numpy as np

In [2]:
root = Path('/data2/project_model_based_fmri/ds001882/')
save_path = Path('.')

## PREPROCESS FMRI IMAGE

In [ ]:
X, masked_data, layout = bids_preprocess(root,save_path=save_path,smoothing_fwhm=None, zoom=(2, 2, 2))

## PREPRROCESS EVENT DATA

In [3]:
def prep_func(row,info):

    row['subjID'] = info['subject']
    row['run'] = info['run']
    
    if row['delay_left'] >= row['delay_right']:
        row['delay_later'] = row['delay_left']
        row['delay_sooner'] = row['delay_right']
        row['amount_later'] = row['money_left']
        row['amount_sooner'] = row['money_right']
        row['choice'] = 1 if row['choice'] == 1 else 0
    else:
        row['delay_later'] = row['delay_right']
        row['delay_sooner'] = row['delay_left']
        row['amount_later'] = row['money_right']
        row['amount_sooner'] = row['money_left']
        row['choice'] = 1 if row['choice'] == 2 else 0
        
    return row

In [4]:
def cond_func(row):
    return row['agent']==0

In [5]:
def latent_func(row,info,k,beta):
    
    ev_later   = row['amount_later'] / (1 + k * row['delay_later'])
    ev_sooner  = row['amount_sooner'] / (1 + k * row['delay_sooner'])
    utility = ev_later - ev_sooner
    row['modulation'] = utility
    
    return row

In [6]:
params_name = ['k','beta']

In [7]:
dm_model = 'dd_hyperbolic'

In [8]:
df_events = None #pd.read_csv(save_path / 'df_events.tsv',sep = '\t',index_col='Unnamed: 0')

In [9]:
all_individual_params = pd.read_csv(save_path / 'all_ind_pars.tsv',sep = '\t',index_col='Unnamed: 0')

In [10]:
dm_model, df_events, signals, time_masks = preprocess_events(root, 
                                                              dm_model=dm_model,
                                                              latent_func=latent_func, 
                                                              params_name=params_name,
                                                              layout=None,
                                                              prep_func=prep_func,
                                                              cond_func=cond_func,
                                                              df_events=df_events,
                                                              all_individual_params=all_individual_params,
                                                              use_duration=False,
                                                              hrf_model='glover',
                                                              save=True,
                                                              save_path=save_path,
                                                              ncore=16)


modulation signal making..                        :  83%|████████▎ | 5/6 [00:29<00:08,  8.31s/it]/home/cheoljun/anaconda3/envs/model_based_fmri/lib/python3.7/site-packages/model_based_mvpa-0.0.1-py3.7.egg/model_based_mvpa/preprocessing/events.py:259: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  signals = np.array(signals)
events preproecssing done!                        : : 7it [00:29,  5.95s/it]                     INFO:root:time elapsed: 0.50 minutes
events preproecssing done!                        : : 7it [00:29,  4.27s/it]


## LOAD DATA

In [11]:
prep_path = save_path
data_path_list = [prep_path / f'X_{i:02d}.npy' for i in range(1,21)]
X = np.concatenate([np.load(data_path) for data_path in data_path_list],0)
X = X.reshape(-1,X.shape[-1])

In [12]:
y = np.load(prep_path / 'y.npy' , allow_pickle=True)
y = np.concatenate(y,0)
X = X.reshape(-1,X.shape[-1])
y = y.flatten()


In [13]:
time_mask = np.load(prep_path / 'y_mask.npy', allow_pickle=True)
time_mask = np.concatenate(time_mask,0)
time_mask = time_mask.flatten()


In [ ]:
X = X[time_mask>0]
y = y[time_mask>0]

masked_data = nib.load(prep_path / 'masked_data.nii.gz')

## FIT MVPA MODEL

In [ ]:
from model_based_mvpa.models.regressor import *
from model_based_mvpa.utils.coef2map import *
import time

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))
N = 30
coefs = mlp_regression(X, y,
                       layer_dims=[1024, 1024],
                       activation_func='linear',
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=30,
                       verbose=1)

task_name = f'piva2019_mlp_{N}'
result = get_map(coefs, masked_data, task_name=task_name, map_type='z', save_path='.', smoothing_sigma=1)
print(time.strftime('%c', time.localtime(time.time())))

In [ ]:
print(time.strftime('%c', time.localtime(time.time())))
N=3
alpha  = 0.0001
coefs = elasticnet(X, y,
               alpha=alpha,
               n_jobs=16,
               N=N,
               verbose=0)

task_name = f'piva2019_elasticnet_{N}_{alpha:.4f}'
result = get_map(coefs, masked_data, task_name=task_name, map_type='z', save_path='.', smoothing_sigma=1)
print(time.strftime('%c', time.localtime(time.time())))